#### The goal of this file is to be able to generate a table similar to the one found in [this video](https://youtu.be/3Zcv9JQBgc8?t=682) at the *11:22* mark. We can use this to see how detrimental it is to a team if they do not score in certain scenarios.   
   * If a team has a **90%+** chance of scoring in a given scenario with **0** outs, but a roughly **50%** chance to score in the same scenario with **1** out, we can begin to explain why a team is struggling to win close games.

In [1]:
import pandas as pd
from pprint import pprint

In [2]:
# updates the count for combinations of baserunners and outs after every inning parsed or after each time a run is scored while parsing
# scored is True if this function is called due to a run scoring after an AB
def update_tally(scenarios, temp_scenarios, scored=False):
    for b in temp_scenarios:
            for o in temp_scenarios[b]:
                scenarios[b][o]["num"] += temp_scenarios[b][o]
                if scored:
                    scenarios[b][o]["scored"] += temp_scenarios[b][o]
                temp_scenarios[b][o] = 0
    return scenarios, temp_scenarios

In [17]:
def find_probs(team):
    data = data = pd.read_excel(f"Team Batting Logs/{team}.xlsx")
    # will hold final tallies of baserunner-out combinations
    scenarios = {}
    # this dict is used to keep track of base-out combos that are seen before we know if a run scored that inning or not
    temp_scenarios = {}
    # used to keep track of inning changes
    curr_inn = None

    for ab in data.iterrows():
        inn = ab[1][0]
        # check if inning ended
        if not curr_inn or curr_inn != inn:
            curr_inn = inn
            scenarios, temp_scenarios = update_tally(scenarios, temp_scenarios)
        # get baserunners and outs
        bases = ab[1][3]
        outs = ab[1][2]
        # initialize combinations to dictionaries- final dict starts at 0 and is only added to when update_tally() is called
        if bases not in scenarios:
            scenarios[bases] = {outs: {"num": 0, "scored": 0}}
            temp_scenarios[bases] = {outs: 1}
        elif outs not in scenarios[bases]:
            scenarios[bases][outs] = {"num": 0, "scored": 0}
            temp_scenarios[bases][outs] = 1
        # if already initialized, add to temp tally
        else:
            temp_scenarios[bases][outs] += 1
        # check if a run scored as a result of AB; if one did, we update the tally
        run_scored = type(ab[1][5]) ==  str and "R" in ab[1][5]
        if run_scored:
            scenarios, temp_scenarios = update_tally(scenarios, temp_scenarios, True)
    # one last update after the end of the sheet
    scenarios, temp_scenarios = update_tally(scenarios, temp_scenarios)
    
    # convert to percentages matrix and transpose + sort matrix for better viewing
    scenarios = {k: {k1: round(100*v1['scored']/v1['num'], 2) for k1,v1 in v.items()} for k,v in scenarios.items()}
    scenarios = pd.DataFrame(scenarios).T
    scenarios.sort_index(inplace=True)
    # final result will look similar to the graphic in the video above
    return scenarios

In [18]:
pprint(find_probs("ATL"))

          0      1      2
---   29.17  16.79   8.58
--3   90.00  65.52  29.73
-2-   66.67  42.86  28.07
-23  100.00  57.14  21.21
1--   39.39  26.02  15.84
1-3   92.86  71.43  35.42
12-   51.28  36.23  26.88
123   86.67  68.57  31.71


As we can see, the *Braves* have a **90%** chance to score with a runner on 3rd and **0** outs, but if they get out and don't score as a result, those odds drop to around **66%**.

In [19]:
pprint(find_probs("SEA"))

         0      1      2
---  25.77  13.25   5.44
--3  75.00  69.23  35.71
-2-  65.00  40.74  25.24
-23  75.00  73.68  18.75
1--  43.14  23.91  10.15
1-3  92.31  54.05  34.29
12-  61.70  39.71  20.41
123  91.67  78.26  37.50


The *Mariners* have it slightly different. Sporting the worst batting average in the MLB at an abysmal **.218**, we can see that they don't necessarily excel with a runner on 3rd and **0** outs, scoring that runner only **75%** of the time. However, even they top the *Braves* with both the bases loaded and having runners at 1st and 2nd. In some cases, they have a run cross the plate at a rate that's almost **10%** higher than the *Braves* do.  
  
This is notable beacuse the scenarios that Seattle excels in happen quite more frequently than the ones Atlanta favors. Even if we counted every time a runner was just on 3rd for both teams, you'd still only have **134** at bats compared to almost **600** with either the bases loaded or a runner on 1st and 2nd for both teams combined.